In [2]:
%cd /content/drive/MyDrive/Recommendation Engine

/content/drive/MyDrive/Recommendation Engine


In [3]:
import pandas as pd
import numpy as np

In [4]:
movies_df = pd.read_csv("./100k/movies.csv")
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [5]:
ratings_df = pd.read_csv("./100k/ratings.csv")
ratings_df


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


## Now combine the two tables and drop things we dont have to use



In [6]:
merged_df_on_movieId = pd.merge(ratings_df, movies_df, on='movieId')


In [7]:
merged_df_on_movieId

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama
...,...,...,...,...,...,...
99999,664,64997,2.5,1343761859,War of the Worlds (2005),Action|Sci-Fi
100000,664,72380,3.5,1344435977,"Box, The (2009)",Drama|Horror|Mystery|Sci-Fi|Thriller
100001,665,129,3.0,995232528,Pie in the Sky (1996),Comedy|Romance
100002,665,4736,1.0,1010197684,Summer Catch (2001),Comedy|Drama|Romance


In [8]:
merged_df_on_movieId = merged_df_on_movieId.drop(['timestamp', 'genres'], axis=1)
merged_df_on_movieId.head()

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)


In [9]:
merged_df_on_movieId.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  int64  
 2   rating   100004 non-null  float64
 3   title    100004 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [10]:

movie_ratingCount = (merged_df_on_movieId.groupby(by = ['title'])['rating'].count().reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})[['title', 'totalRatingCount']])
movie_ratingCount.head(10)

,title,totalRatingCount
0,"""Great Performances"" Cats (1998)",2
1,$9.99 (2008),3
2,'Hellboy': The Seeds of Creation (2004),1
3,'Neath the Arizona Skies (1934),1
4,'Round Midnight (1986),2
5,'Salem's Lot (2004),1
6,'Til There Was You (1997),4
7,"'burbs, The (1989)",19
8,'night Mother (1986),3
9,(500) Days of Summer (2009),45


In [11]:
movie_ratingCount

,title,totalRatingCount
0,"""Great Performances"" Cats (1998)",2
1,$9.99 (2008),3
2,'Hellboy': The Seeds of Creation (2004),1
3,'Neath the Arizona Skies (1934),1
4,'Round Midnight (1986),2
...,...,...
9059,xXx (2002),23
9060,xXx: State of the Union (2005),1
9061,¡Three Amigos! (1986),31
9062,À nous la liberté (Freedom for Us) (1931),1


In [12]:
rating_with_totalRatingCount = merged_df_on_movieId.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,31,2.5,Dangerous Minds (1995),42
1,7,31,3.0,Dangerous Minds (1995),42
2,31,31,4.0,Dangerous Minds (1995),42
3,32,31,4.0,Dangerous Minds (1995),42
4,36,31,3.0,Dangerous Minds (1995),42
5,39,31,3.0,Dangerous Minds (1995),42
6,73,31,3.5,Dangerous Minds (1995),42
7,88,31,3.0,Dangerous Minds (1995),42
8,96,31,2.5,Dangerous Minds (1995),42
9,110,31,4.0,Dangerous Minds (1995),42


## Now drop the duplicate data


In [13]:
user_rating = rating_with_totalRatingCount.drop_duplicates(['userId','title'])
user_rating.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,31,2.5,Dangerous Minds (1995),42
1,7,31,3.0,Dangerous Minds (1995),42
2,31,31,4.0,Dangerous Minds (1995),42
3,32,31,4.0,Dangerous Minds (1995),42
4,36,31,3.0,Dangerous Minds (1995),42
5,39,31,3.0,Dangerous Minds (1995),42
6,73,31,3.5,Dangerous Minds (1995),42
7,88,31,3.0,Dangerous Minds (1995),42
8,96,31,2.5,Dangerous Minds (1995),42
9,110,31,4.0,Dangerous Minds (1995),42


In [14]:
user_rating.shape

(100003, 5)

In [34]:
user_rating

,userId,movieId,rating,title,totalRatingCount
0,1,31,2.5,Dangerous Minds (1995),42
1,7,31,3.0,Dangerous Minds (1995),42
2,31,31,4.0,Dangerous Minds (1995),42
3,32,31,4.0,Dangerous Minds (1995),42
4,36,31,3.0,Dangerous Minds (1995),42
...,...,...,...,...,...
99998,664,60832,3.0,Pathology (2008),1
100000,664,72380,3.5,"Box, The (2009)",1
100001,665,129,3.0,Pie in the Sky (1996),1
100002,665,4736,1.0,Summer Catch (2001),1


## Matrix Factorization


### Now create a matrix and fill 0 values


In [1]:
# Item Based Filtering

In [15]:
movie_user_rating_pivot = user_rating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
movie_user_rating_pivot

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X = movie_user_rating_pivot.values.T #extracted the values
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
X.shape

(9064, 671)

In [18]:
Y = movie_user_rating_pivot.values
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
Y.shape

(671, 9064)

## Now lets fit the model using Singular Value Decomposition


## Item Based Filtering

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD #Imported truncated singular value Decomposition which reduces the dimention of the matrix

SVD = TruncatedSVD(n_components=12, random_state=17) # intantiated SVD with the number of columns finally
matrix = SVD.fit_transform(X) # fit and transform
matrix.shape

(9064, 12)

In [ ]:
matri

In [ ]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)  # Getting the correlation coefficients
corr.shape

(9064, 9064)

In [ ]:
movie_title = movie_user_rating_pivot.columns # To get all the movies
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)") # extracting the index of the movie we wanted to get the recommendations for

In [ ]:
corr_coffey_hands  = corr[coffey_hands] # Extracting the correlation coefficients of other movies based on the movie we are searching
list(movie_title[(corr_coffey_hands >= 0.95)])


['Abduction (2011)',
 'Adventures of Tintin, The (2011)',
 'Amazing Spider-Man, The (2012)',
 'Avengers, The (2012)',
 'Big Hero 6 (2014)',
 'Captain America: The First Avenger (2011)',
 'Dark Knight Rises, The (2012)',
 'Diary of a Wimpy Kid (2010)',
 'Drillbit Taylor (2008)',
 'Edge of Tomorrow (2014)',
 'Elysium (2013)',
 'Ex Machina (2015)',
 'Ghost Rider: Spirit of Vengeance (2012)',
 'Gravity (2013)',
 'Green Hornet, The (2011)',
 'Guardians of the Galaxy (2014)',
 'Hobbit: An Unexpected Journey, The (2012)',
 'In Time (2011)',
 'Indie Game: The Movie (2012)',
 'Iron Man 2 (2010)',
 'Iron Man 3 (2013)',
 'Limitless (2011)',
 'Pacific Rim (2013)',
 'Paperboy, The (2012)',
 'Resident Evil: Retribution (2012)',
 'Rio (2011)',
 'Skyfall (2012)',
 'Star Trek Into Darkness (2013)',
 'Star Wars: Episode VII - The Force Awakens (2015)',
 'TMNT (Teenage Mutant Ninja Turtles) (2007)',
 'The Hobbit: The Battle of the Five Armies (2014)',
 'The Hunger Games (2012)',
 'Thor (2011)',
 'Tron: L

## User Based Filtering

In [21]:
import sklearn
from sklearn.decomposition import TruncatedSVD #Imported truncated singular value Decomposition which reduces the dimention of the matrix

SVD = TruncatedSVD(n_components=12, random_state=17) # intantiated SVD with the number of columns finally
user_matrix = SVD.fit_transform(Y) # fit and transform 
user_matrix.shape

(671, 12)

In [22]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(user_matrix)  # Getting the correlation coefficients
corr.shape

(671, 671)

In [37]:
user_ids = movie_user_rating_pivot.T.columns
user_id_list = list(user_ids)
user_index = user_id_list.index(10) # You can enter the user id of your choice

In [39]:
user_specific_corr  = corr[user_index] # Extracting the correlation coefficients of other movies based on the movie we are searching
list(user_ids[(user_specific_corr >= 0.9)])

[10, 66, 74, 155, 202, 214, 218, 271, 291, 368, 651]